In [1]:
%%bash

sudo systemctl stop greengrass.service
sudo systemctl disable greengrass.service
sudo rm /etc/systemd/system/greengrass.service
sudo systemctl daemon-reload && sudo systemctl reset-failed
sudo rm -rf /greengrass/v2

Failed to stop greengrass.service: Unit greengrass.service not loaded.
Failed to disable unit: Unit file greengrass.service does not exist.
rm: cannot remove '/etc/systemd/system/greengrass.service': No such file or directory


### Get the Greengrass software on the Pupper

Download and unzip the Greengrass core software.

In [2]:
!echo "downloading" && curl -s https://d2s8p88vqu9w66.cloudfront.net/releases/greengrass-nucleus-latest.zip > greengrass-nucleus-latest.zip && unzip -o greengrass-nucleus-latest.zip -d GreengrassCore && echo "done"

downloading
Archive:  greengrass-nucleus-latest.zip
  inflating: GreengrassCore/META-INF/MANIFEST.MF  
  inflating: GreengrassCore/META-INF/SIGNER.SF  
  inflating: GreengrassCore/META-INF/SIGNER.RSA  
  inflating: GreengrassCore/LICENSE  
  inflating: GreengrassCore/NOTICE   
  inflating: GreengrassCore/README.md  
  inflating: GreengrassCore/THIRD-PARTY-LICENSES  
  inflating: GreengrassCore/bin/greengrass.exe  
  inflating: GreengrassCore/bin/greengrass.service.procd.template  
  inflating: GreengrassCore/bin/greengrass.service.template  
  inflating: GreengrassCore/bin/greengrass.xml.template  
  inflating: GreengrassCore/bin/loader  
  inflating: GreengrassCore/bin/loader.cmd  
  inflating: GreengrassCore/conf/recipe.yaml  
  inflating: GreengrassCore/lib/Greengrass.jar  
done


### Get the CA Certificate

Download the CA certificate.

In [5]:
!echo "downloading ca certificate" && sudo curl -s https://www.amazontrust.com/repository/AmazonRootCA1.pem > AmazonRootCA1.pem && echo "downloaded ca certificate"

downloading ca certificate
downloaded ca certificate


### Register the Pupper with AWS using Greengrass
Run the Greengrass provisioning command.
You need to paste in your AWS credentials from the workshop authentication page or IAM user.
Change aws-region, thing-name, and thing-group-name accordingly.

In [2]:
%%bash

export AWS_ACCESS_KEY_ID=""
export AWS_SECRET_ACCESS_KEY=""
export AWS_SESSION_TOKEN="
"

sudo -E java -Droot="/greengrass/v2" -Dlog.store=FILE -jar ~/GreengrassCore/lib/Greengrass.jar \
--aws-region us-west-2 \
--thing-name mini-pupper-1 \
--thing-group-name mini-pupper \
--component-default-user ggc_user:ggc_group \
--provision true \
--setup-system-service true \
--deploy-dev-tools true

Creating user ggc_user 
ggc_user created 
Creating group ggc_group 
ggc_group created 
Added ggc_user to ggc_group 
Provisioning AWS IoT resources for the device with IoT Thing Name: [mini-pupper-3]...
Found IoT policy "GreengrassV2IoTThingPolicy", reusing it
Creating keys and certificate...
Attaching policy to certificate...
Creating IoT Thing "mini-pupper-3"...
Attaching certificate to IoT thing...
Successfully provisioned AWS IoT resources for the device with IoT Thing Name: [mini-pupper-3]!
Adding IoT Thing [mini-pupper-3] into Thing Group: [mini-pupper]...
IoT Thing Group "mini-pupper" already existed, reusing it
Successfully added Thing into Thing Group: [mini-pupper]
Setting up resources for aws.greengrass.TokenExchangeService ... 
Attaching TES role policy to IoT thing...
No managed IAM policy found, looking for user defined policy...
IAM policy named "GreengrassV2TokenExchangeRoleAccess" already exists. Please attach it to the IAM role if not already
Configuring Nucleus with p

### Empower ggc_user to make full use of the Pupper's systems

Give the newly created ggc_user access to run docker applications and the i2c bus:

In [3]:
%%bash

sudo usermod -aG docker ggc_user
sudo usermod -aG i2c ggc_user
sudo usermod -aG video ggc_user
sudo setfacl --modify group:docker:rw /var/run/docker.sock
echo "done"

done


### Check the Pupper has appeared in the AWS Console

In a new browser tab, open the Greengrass Console and click on Core Devices. You should see your new Greengrass application running!

### Use the Greengrass CLI to see what the Pupper is capable of

Note: The CLI deployment takes ~4 minutes, if you see errors here, wait a few minutes and try again.

Run the following command to interact with the Greengrass CLI on the robot:

In [1]:
%%bash

cd /greengrass/v2/bin
sudo ./greengrass-cli component list

Components currently running in Greengrass:
Component Name: DeploymentService
    Version: 0.0.0
 State: RUNNING
    Configuration: null
Component Name: FleetStatusService
    Version: 0.0.0
    State: RUNNING
    Configuration: null
Component Name: aws.greengrass.Nucleus
    Version: 2.14.1
ED  State: FINISH
    Configuration: {"awsRegion":"us-west-2","componentStoreMaxSizeBytes":"10000000000","deploymentPollingFrequencySeconds":"15","envStage":"prod","fipsMode":"false","fleetStatus":{"periodicStatusPublishIntervalSeconds":86400.0},"greengrassDataPlaneEndpoint":"","greengrassDataPlanePort":"8443","httpClient":{},"iotCredEndpoint":"c2p23f7ik8hx6z.credentials.iot.us-west-2.amazonaws.com","iotDataEndpoint":"a2bv3hsgw5zaxi-ats.iot.us-west-2.amazonaws.com","iotRoleAlias":"GreengrassV2TokenExchangeRoleAlias","jvmOptions":"-Dlog.store=FILE","logging":{},"mqtt":{"spooler":{}},"networkProxy":{"proxy":{}},"platformOverride":{},"runWithDefault":{"posixShell":"sh","posixUser":"ggc_user:ggc_group"

### Done!

You can now return to the workshop documentation.